In [48]:
#FN score he got was around 90% in macro, more then 80
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
import matplotlib.pyplot as plot

# we can use the LabelEncoder to encode the gender feature
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV, cross_validate

# importing two different imputation methods that take into consideration all the features when predicting the missing values
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.dummy import DummyClassifier #Will identify the maority calss base line, model needs to do better then the baseline

# oversample the minority class using SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter

# to reduce randomness then you put the seed
np.random.seed(42)

In [49]:
df = pd.read_csv("./data/student_prediction.csv")
df.rename(columns = {'KIDS':'PARENT_STATUS'}) #There is a column name error in the data noted in the Kaggle description, this fixes it.
df = df.drop(["STUDENTID"], axis=1)
print(f"Data shape: \n{df.shape}\n")
print(f"Data size: \n{df.size}\n")
print(f"Data ndim: \n{df.ndim}\n")
print("_____________________________________________\n")

data_train, data_test = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train Data shape: \n{data_train}\n")
print(f"Test Data shape: \n{data_test}\n")


Data shape: 
(145, 32)

Data size: 
4640

Data ndim: 
2

_____________________________________________

Train Data shape: 
     AGE  GENDER  HS_TYPE  SCHOLARSHIP  WORK  ACTIVITY  PARTNER  SALARY  \
9      2       1        2            3     2         2        1       3   
4      2       2        1            3     2         2        1       3   
26     2       2        2            3     2         1        1       1   
120    2       1        1            3     1         1        1       2   
125    1       1        2            5     1         1        2       1   
..   ...     ...      ...          ...   ...       ...      ...     ...   
71     1       1        3            4     2         2        2       1   
106    1       2        2            4     2         1        2       1   
14     3       2        2            4     1         1        2       3   
92     1       2        2            3     2         2        2       1   
102    1       2        2            3     2        

In [50]:
df

,AGE,GENDER,HS_TYPE,SCHOLARSHIP,WORK,ACTIVITY,PARTNER,SALARY,TRANSPORT,LIVING,...,PREP_STUDY,PREP_EXAM,NOTES,LISTENS,LIKES_DISCUSS,CLASSROOM,CUML_GPA,EXP_GPA,COURSE ID,GRADE
0,2,2,3,3,1,2,2,1,1,1,...,1,1,3,2,1,2,1,1,1,1
1,2,2,3,3,1,2,2,1,1,1,...,1,1,3,2,3,2,2,3,1,1
2,2,2,2,3,2,2,2,2,4,2,...,1,1,2,2,1,1,2,2,1,1
3,1,1,1,3,1,2,1,2,1,2,...,1,2,3,2,2,1,3,2,1,1
4,2,2,1,3,2,2,1,3,1,4,...,2,1,2,2,2,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2,1,2,3,1,1,2,1,1,2,...,1,1,2,1,2,1,3,3,9,5
141,1,1,2,4,2,2,2,1,4,2,...,1,1,3,2,2,1,5,3,9,5
142,1,1,1,4,2,2,2,1,1,1,...,1,1,3,3,2,1,4,3,9,1
143,2,1,2,4,1,1,1,5,2,3,...,2,1,2,1,2,1,5,3,9,4


In [51]:
#Extracting Labels

columns = data_train.columns.to_list()
columns_drop = columns.pop(-1)
labels_train = data_train.drop(columns, axis=1)
labels_test = data_test.drop(columns, axis=1)

print(f"labels_train: \n{labels_train}\n")
print(f"labels_test: \n{labels_test}\n")

features_train = data_train.drop(['GRADE'], axis=1)
features_test = data_test.drop(['GRADE'], axis=1)
print(f"features_train: \n{features_train }\n")
print(f"lfeatures_test: \n{features_test }\n")



labels_train: 
     GRADE
9        0
4        1
26       1
120      1
125      2
..     ...
71       6
106      7
14       2
92       7
102      7

[116 rows x 1 columns]

labels_test: 
     GRADE
69       5
140      5
27       1
19       3
42       1
117      1
126      3
108      6
84       7
18       2
12       0
55       3
128      0
78       4
73       6
36       2
112      2
133      3
100      6
101      7
94       6
136      0
11       0
66       5
31       3
45       3
51       4
76       4
111      2

features_train: 
     AGE  GENDER  HS_TYPE  SCHOLARSHIP  WORK  ACTIVITY  PARTNER  SALARY  \
9      2       1        2            3     2         2        1       3   
4      2       2        1            3     2         2        1       3   
26     2       2        2            3     2         1        1       1   
120    2       1        1            3     1         1        1       2   
125    1       1        2            5     1         1        2       1   
..   ...     ...

In [52]:
num_pipeline = Pipeline([
        ('imputer', KNNImputer(n_neighbors=5)),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(features_train)


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs)
    ])

data_prepared = pd.DataFrame(full_pipeline.fit_transform(features_train),columns=features_train.columns, index=features_train.index)
#full_pipeline.fit_transform(data_train)
#pd.DataFrame(full_pipeline.fit_transform(train_df),columns=train_df.columns, index=train_df.index)
print(data_prepared)

         AGE    GENDER   HS_TYPE  SCHOLARSHIP      WORK  ACTIVITY   PARTNER  \
9    0.67735 -1.302948  0.051793    -0.703008  0.684364  0.810643 -1.190238   
4    0.67735  0.767490 -1.950887    -0.703008  0.684364  0.810643 -1.190238   
26   0.67735  0.767490  0.051793    -0.703008  0.684364 -1.233588 -1.190238   
120  0.67735 -1.302948 -1.950887    -0.703008 -1.461210 -1.233588 -1.190238   
125 -1.03075 -1.302948  0.051793     1.731288 -1.461210 -1.233588  0.840168   
..       ...       ...       ...          ...       ...       ...       ...   
71  -1.03075 -1.302948  2.054474     0.514140  0.684364  0.810643  0.840168   
106 -1.03075  0.767490  0.051793     0.514140  0.684364 -1.233588  0.840168   
14   2.38545  0.767490  0.051793     0.514140 -1.461210 -1.233588  0.840168   
92  -1.03075  0.767490  0.051793    -0.703008  0.684364  0.810643  0.840168   
102 -1.03075  0.767490  0.051793    -0.703008  0.684364  0.810643 -1.190238   

       SALARY  TRANSPORT    LIVING  ...    ATTEND  

In [53]:
# create the following four different models with their default hyperparameter values to be trained using the preprocessed data (0.5 * 4)
labelTrainFlat = labels_train.values.ravel()
# Gradient Boosting
gradientBoosting = GradientBoostingClassifier()
gradientBoosting = gradientBoosting.fit(data_prepared, labelTrainFlat)

# Decision Trees
decisionTree = DecisionTreeClassifier()
decisionTree = decisionTree.fit(data_prepared,labelTrainFlat)

# Random Forests
randomForest = RandomForestClassifier()
randomForest = randomForest.fit(data_prepared,labelTrainFlat)

In [54]:
parametersGradientBoosting = [
    {'n_estimators': [10,100,50],'min_samples_split': [2,3], 'min_samples_leaf': [2,4,5]}
]

parametersDecisionTree = [
    {'max_depth': [1,2,3,4],'min_samples_split': [2,3], 'min_samples_leaf': [4,5]}
]

parametersRandomForest = [
    {'n_estimators': [145,150,155],'max_depth': [5,6,7], 'bootstrap': [True, False]}
]

# initialize gridsearch with the required parameters, including the following scoring methods and refit='bal_accuracy' (2)
scoringX = {"accuracy": "accuracy", "bal_accuracy": "balanced_accuracy", "F1_macro": "f1_macro"}

grid_searchGradientBoosting = GridSearchCV(gradientBoosting, parametersGradientBoosting, cv=5, scoring = scoringX, return_train_score=True, n_jobs=-1, refit='bal_accuracy')

grid_searchDecisionTree = GridSearchCV(decisionTree, parametersDecisionTree, cv=5, scoring = scoringX, return_train_score=True, n_jobs=-1, refit='bal_accuracy')

grid_searchRandomForest = GridSearchCV(randomForest, parametersRandomForest, cv=5, scoring = scoringX, return_train_score=True, n_jobs=-1, refit='bal_accuracy')



# fit the training data (0.5)
grid_searchGradientBoosting.fit(data_prepared, labelTrainFlat)
grid_searchDecisionTree.fit(data_prepared, labelTrainFlat)
grid_searchRandomForest.fit(data_prepared, labelTrainFlat)

# print the best parameters (0.5)
print(f"Best parameters GradientBoosting: \n{grid_searchGradientBoosting.best_params_}\n")
print(f"Best parameters DecisionTree: \n{grid_searchDecisionTree.best_params_}\n")
print(f"Best parameters RandomForest: \n{grid_searchRandomForest.best_params_}\n")

print(f"Best estimator GradientBoosting: \n{grid_searchGradientBoosting.best_estimator_}\n")
print(f"Best estimator DecisionTree: \n{grid_searchDecisionTree.best_estimator_}\n")
print(f"Best estimator RandomForest: \n{grid_searchRandomForest.best_estimator_}\n")

print(f"Best score GradientBoosting: \n{grid_searchGradientBoosting.best_score_}\n")
print(f"Best score DecisionTree: \n{grid_searchDecisionTree.best_score_}\n")
print(f"Best score RandomForest: \n{grid_searchRandomForest.best_score_}\n")

C:\Users\adamj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\adamj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\adamj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best parameters GradientBoosting: 
{'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 10}

Best parameters DecisionTree: 
{'max_depth': 4, 'min_samples_leaf': 5, 'min_samples_split': 3}

Best parameters RandomForest: 
{'bootstrap': True, 'max_depth': 6, 'n_estimators': 145}

Best estimator GradientBoosting: 
GradientBoostingClassifier(min_samples_leaf=2, n_estimators=10)

Best estimator DecisionTree: 
DecisionTreeClassifier(max_depth=4, min_samples_leaf=5, min_samples_split=3)

Best estimator RandomForest: 
RandomForestClassifier(max_depth=6, n_estimators=145)

Best score GradientBoosting: 
0.3698979591836734

Best score DecisionTree: 
0.2968962585034013

Best score RandomForest: 
0.26258503401360545



In [55]:
# print the grid search cross-validation results listing the above mentioned evaluation methods (3)
cross_val_resultsGB = grid_searchGradientBoosting.cv_results_
cross_val_resultsDT = grid_searchDecisionTree.cv_results_
cross_val_resultsRF = grid_searchRandomForest.cv_results_



print(f"Mean Test Accuracy for GradientBoosting: \n{cross_val_resultsGB['mean_test_accuracy']}\n")
print(f"Balanced Test Accuracy for GradientBoosting: \n{cross_val_resultsGB['mean_test_bal_accuracy']}\n")
print(f"Mean F1 Macro for GradientBoosting: \n{cross_val_resultsGB['mean_test_F1_macro']}\n")

#DTC
print(f"Mean Test Accuracy for Decision Trees: \n{cross_val_resultsDT['mean_test_accuracy']}\n")
print(f"Balanced Test Accuracy for Decision Trees: \n{cross_val_resultsDT['mean_test_bal_accuracy']}\n")
print(f"Mean F1 Macro for Decision Trees: \n{cross_val_resultsDT['mean_test_F1_macro']}\n")

#RFC
print(f"Mean Test Accuracy for Random Forests: \n{cross_val_resultsRF['mean_test_accuracy']}\n")
print(f"Balanced Test Accuracy for Random Forests: \n{cross_val_resultsRF['mean_test_bal_accuracy']}\n")
print(f"Mean F1 Macro for Random Forests: \n{cross_val_resultsRF['mean_test_F1_macro']}\n")

#NB


Mean Test Accuracy for GradientBoosting: 
[0.44855072 0.32789855 0.33695652 0.42355072 0.32826087 0.31992754
 0.38007246 0.3192029  0.32753623 0.38007246 0.31086957 0.3365942
 0.39637681 0.31050725 0.30181159 0.38804348 0.31050725 0.31884058]

Balanced Test Accuracy for GradientBoosting: 
[0.36989796 0.23954082 0.26305272 0.3565051  0.23745748 0.25680272
 0.31237245 0.23329082 0.25412415 0.30820578 0.22971939 0.26335034
 0.34787415 0.25569728 0.24221939 0.34013605 0.25301871 0.25829082]

Mean F1 Macro for GradientBoosting: 
[0.36214994 0.22683114 0.24699877 0.34981443 0.21740306 0.23962288
 0.31155038 0.20989441 0.23321656 0.30301313 0.20782466 0.24254955
 0.34465449 0.22869554 0.2159759  0.33071339 0.22484277 0.23003228]

Mean Test Accuracy for Decision Trees: 
[0.31884058 0.31884058 0.31884058 0.31884058 0.4134058  0.4134058
 0.42210145 0.42210145 0.35362319 0.36231884 0.37101449 0.37971014
 0.31050725 0.31050725 0.37971014 0.3884058 ]

Balanced Test Accuracy for Decision Trees: 
[0.